In [2]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from functools import partial
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [4]:
nrows = 1000000
train = prepare_data('train', nrows=nrows, recompute=False)
df = compute_session_fts(train, 'train')

[05-25 12:00:56 - create_model_inputs-48 - prepare_data - INFO] Load from existing ./nn_cache/train_1000000_no_test.snappy
[05-25 12:01:33 - create_model_inputs-158 - compute_session_fts - INFO] Session features generated: ['timestamp_session_duration', 'timestamp_dwell_time_prior_clickout', 'current_filters_last_filters', 'session_id_size']


In [6]:
df = df.groupby('session_id').last().reset_index()
# df['session_id_size'] = np.log(df['session_id_size'])
# df['cfs'] = df['current_filters_last_filters'].str.lower().str.split('|')

# df['prices'] = df['prices'].str.split('|')
# df['prices'] = df['prices'].apply(lambda x: [int(p) for p in x])

# df['time_steps'] = df['prices'].str.len()
# padding_mask = df['time_steps'] < 25
# df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
#                                                                                        mode='constant'))
df['imps'] = df['impressions'].str.split('|')
df['imps'] = df['imps'].apply(lambda x: [int(i) for i in x])
# df.loc[padding_mask, 'impressions'] = (df.loc[padding_mask, 'impressions']
#                                          .apply(lambda x: np.pad(x, (0, 25-len(x)),mode='constant')))
df['reference'] = df['reference'].astype(int)
# filter out nan rows with reference_id not in impressions list, since if the true target in test
# is not in the impression list then it would not get evaluated
def assign_target(row):
    ref = row['reference']
    imp = list(row['imps'])
    if ref in imp:
        return imp.index(ref)
    else:
        return np.nan

df['target'] = df.apply(assign_target, axis=1)
# drop the ones whose reference is not in the impression list
df = df[df['target'].notna()].reset_index(drop=True)
df['target'] = df['target'].astype(int)

### comapre session with size 1 and the rest

In [7]:
sids = df['session_id'].unique()

---
### ones

In [8]:
display_cols = ['session_id', 'step', 'action_type', 'current_filters', 'reference', 'timestamp',
                'impressions', 'prices', 'target', 'session_id_size']
raw_display_cols = [c for c in train.columns if c in display_cols]

In [9]:
# pd.options.display.max_colwidth = 100

In [11]:
rid = np.random.choice(sids, 1)[0]
print(rid)
df[df.session_id==rid][display_cols].reset_index(drop=True)

8e91e2211be60


,session_id,step,action_type,current_filters,reference,timestamp,impressions,prices,target,session_id_size
0,8e91e2211be60,1,clickout item,None,834876,2018-11-05 23:30:10,834876|8796|1811137|67445|7554016|8815|8750|87...,210|145|166|283|199|192|259|259|548|226|334|29...,0,1


In [12]:
pp.pprint(df[df.session_id==rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

{'action_type': ['clickout item'],
 'current_filters': [None],
 'impressions': ['834876|8796|1811137|67445|7554016|8815|8750|8758|8766|8788|8837|61684|149598|7304664|8985292|1436082|8724|8809|1329920|4916380|2891353|8744|5724812|6066134|8812'],
 'prices': ['210|145|166|283|199|192|259|259|548|226|334|295|138|245|182|248|228|259|224|259|259|210|170|133|259'],
 'reference': [834876],
 'session_id': ['8e91e2211be60'],
 'session_id_size': [1],
 'step': [1],
 'target': [0],
 'timestamp': [Timestamp('2018-11-05 23:30:10')]}


In [13]:
# and the raw
train[train.session_id==rid][raw_display_cols].reset_index(drop=True)

,session_id,timestamp,step,action_type,current_filters,reference,impressions,prices
0,8e91e2211be60,2018-11-05 23:30:10,1,clickout item,None,834876,834876|8796|1811137|67445|7554016|8815|8750|87...,210|145|166|283|199|192|259|259|548|226|334|29...


In [14]:
pp.pprint(train[train.session_id==rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

{'action_type': ['clickout item'],
 'current_filters': [None],
 'impressions': ['834876|8796|1811137|67445|7554016|8815|8750|8758|8766|8788|8837|61684|149598|7304664|8985292|1436082|8724|8809|1329920|4916380|2891353|8744|5724812|6066134|8812'],
 'prices': ['210|145|166|283|199|192|259|259|548|226|334|295|138|245|182|248|228|259|224|259|259|210|170|133|259'],
 'reference': ['834876'],
 'session_id': ['8e91e2211be60'],
 'step': [1],
 'timestamp': [Timestamp('2018-11-05 23:30:10')]}


---
### the generated train input

In [16]:
!ls -lthr ./nn_cache/

total 9153280
-rw-r--r--  1 sam_dessa  staff    18M 25 May 10:47 preprocessed_train.snappy
-rw-r--r--  1 sam_dessa  staff    17M 25 May 10:47 train_1000000_no_test.snappy
-rw-r--r--  1 sam_dessa  staff   1.2G 25 May 10:49 meta_mapping.npy
-rw-r--r--  1 sam_dessa  staff    28K 25 May 10:50 filters_mapping.npy
-rw-r--r--  1 sam_dessa  staff   1.5G 25 May 10:50 train_impressions.npy
-rw-r--r--  1 sam_dessa  staff   9.9M 25 May 10:50 train_prices.npy
-rw-r--r--  1 sam_dessa  staff   9.9M 25 May 10:50 train_prices_rank.npy
-rw-r--r--  1 sam_dessa  staff   1.5G 25 May 10:50 train_merged.npy
-rw-r--r--  1 sam_dessa  staff    81M 25 May 10:50 train_cfilters.npy
-rw-r--r--  1 sam_dessa  staff   2.4M 25 May 10:50 train_numerics.npy
-rw-r--r--  1 sam_dessa  staff   406K 25 May 10:50 train_targets.npy


In [ ]:
tids = np.load('./nn_cache/tra')